In [ ]:
import bgk
import matplotlib.pyplot as plt
import bgk.run_params as rp
import os

In [ ]:
# Parameters to adjust figures
case = "exact" # exact, max, exact-rev, or max-rev
cont = False

nframes = 100
param = rp.ne

In [ ]:
basepath = f"/mnt/lustre/IAM851/jm1667/psc-runs/case1/trials/{case}/"
# paths = ["B00.25-n256", "B00.25-n256-nicell400", "B00.25-n512-cont", "B00.25-n512-nicell25"]
# Bs = [.25] * 4
# ress = [256, 256, 512, 512]
# maxsteps = [1000000, 1000000, 2000000, 2000000]
# from_ymls = [False, False, False, False]
# nicells = [100, 400, 100, 25]
paths = ["B00.10-n256", "B00.10-n512", "B00.10-n1024", "B00.10-n2048"]
Bs = [.1] * 4
ress = [256, 512, 1024, 2048]
maxsteps = [1000000, 1000000, 2000000, 2000000]
from_ymls = [False, False, True, True]
nicells = [100, 100, 100, 100]

In [ ]:
# path_fmt = f"/mnt/lustre/IAM851/jm1667/psc-runs/case1/trials/{case}/B{{:05.2f}}-n{{}}/"

def get_line(path, res, maxstep, nframes, param) -> tuple[list[float], list[float]]:
    # path = path_fmt.format(B, res)

    loader = bgk.Loader(path, engine="pscadios2", species_names=['e', 'i'], max_step=maxstep)
    videoMaker = bgk.VideoMaker(nframes, loader)

    wholeSlice = bgk.DataSlice(slice(None, None), "")
    videoMaker.loadData(param)
    videoMaker.setSlice(wholeSlice)
    
    return videoMaker.times, videoMaker._getMeansAtOrigin(res//128)

In [ ]:
# Load Data
# liness: list[list[tuple]] = []
lines: list[tuple] = []
for halfpath, res, maxstep, from_yml in zip(paths, ress, maxsteps, from_ymls):
    if from_yml:
        print(f"Loading {halfpath} from yaml...")
        data = bgk.autofigs.figdata.OriginMeans(halfpath, param.name)
        x, y = data.times, data.means
        if maxstep:
            x = x[:maxstep]
            y = y[:maxstep]
    else:   
        path = basepath + halfpath
        print(f"Loading {path} from pfd...")
        x, y = get_line(path, res, maxstep, nframes, param)
    lines.append((x, y))

In [ ]:
# Generate Figure
fig, ax = plt.subplots()

ax.set_xlabel("Time")
ax.set_ylabel(f"Mean {param.title}")
# ax.set_title(f"Comparisons of Mean {param.title} Near Origin")
ax.set_title(f"Comparisons of Mean {param.title} Near Origin for $B_0={.25}$")

for B, res, (x, y), nicell in zip(Bs, ress, lines, nicells):
    # ax.plot(x, y, label=f"$B_0={B}, n={res}$")
    
    color = "b" if res == 256 else "r"
    line = "-" if nicell == 100 else "--"
    ax.plot(x, y, label=f"${res}^2, nicell={nicell}$", color=color, ls=line)
    print()


ax.legend();

In [ ]:
def get_fig_path(outdir, fig_name: str) -> str:
    return os.path.join(outdir, fig_name)

def get_fig_name() -> str:
    param_str = param.title.replace("_", "").replace("$", "")
    Bs_str = ",".join([f"B{B:05.2f}" for B in set(Bs)])
    return f"scaling-{param_str}-{case}-{Bs_str}.png"

In [ ]:
# Save Figure
outdir = f"/mnt/lustre/IAM851/jm1667/psc-scrap/figs-{case}-etc/"
os.makedirs(outdir, exist_ok=True)

fig.savefig(get_fig_path(outdir, get_fig_name()), bbox_inches="tight", pad_inches=0.01, dpi=300)